## Purpose
 - We aim to fit a proper model for determining a subject's risk at coronary heart disease in last 10 years (`TenYearCHD`= 1 for YES, and 0 for NO).
 - We will apply feature selection methods such as principal component analysis (PCA), LASSO and elastic net to the logistic regression.
        (1) For PCA: we determine the principal components(PCs) and regard these PCs as the dependent variables in the logistic regression.
        (2) For LASSO: we use L1 regularization technique on the logistic regression.
        (3) For elastic net: we use both L1 and L2 regularization technique on the logistic regression.
 - In addition to PCA and regularization technique, we conduct KMeans and Support Vector Machine (SVM) in order to classify the target variable `TenYearCHD`. 

### Required packages

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import svm

### Data resource
[Framingham Heart Decease Dataset](https://www.kaggle.com/amanajmera1/framingham-heart-study-dataset)
### Load data

In [7]:
df = pd.read_csv('data/framingham.csv')

In [8]:
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [9]:
df.columns

Index(['male', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds',
       'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP',
       'diaBP', 'BMI', 'heartRate', 'glucose', 'TenYearCHD'],
      dtype='object')

### Data proprocessing: remove missing values
First of all, we check whether there're missing values in each variable.

In [5]:
df.isnull().any() 

male               False
age                False
education           True
currentSmoker      False
cigsPerDay          True
BPMeds              True
prevalentStroke    False
prevalentHyp       False
diabetes           False
totChol             True
sysBP              False
diaBP              False
BMI                 True
heartRate           True
glucose             True
TenYearCHD         False
dtype: bool

In [10]:
df['full_cnt'] = df.apply(lambda x: x.count(), axis=1) 
df_cdh = df[(df['full_cnt'] == 16)] # pick up rows with full values
df_cdh.isnull().any()

male               False
age                False
education          False
currentSmoker      False
cigsPerDay         False
BPMeds             False
prevalentStroke    False
prevalentHyp       False
diabetes           False
totChol            False
sysBP              False
diaBP              False
BMI                False
heartRate          False
glucose            False
TenYearCHD         False
full_cnt           False
dtype: bool

In [11]:
df_x = df_cdh.drop(['TenYearCHD','full_cnt'], axis=1)
df_y = df_cdh['TenYearCHD']

### Split data for 70% train and 30% test samples 

In [229]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.3, random_state=100)

### Principal component analysis (PCA)

We denote *PC* as a principal component. Assume that we find $k$ principal components, each of them is composed by the linear combination of the standardized $X_{i}$ ($Z_{i}$).
$$PC_j=\sum_{j=1}^{k}\sum_{i=1}^{15}a_{i,j}Z_{i}$$



In [13]:
# standardizing the original features
x_train_sd = StandardScaler().fit_transform(x_train.values)
x_test_sd = StandardScaler().fit_transform(x_test.values)

In [14]:
pd.DataFrame(data = x_train_sd, columns = x_train.columns).head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
0,-0.880518,-0.875261,0.021795,1.033369,-0.576079,-0.18532,-0.086472,-0.671749,-0.17253,-0.140265,-0.381025,-0.247836,-0.210725,1.176891,-0.204725
1,-0.880518,-1.580613,-0.957072,1.033369,-0.323060,-0.18532,-0.086472,1.488651,-0.17253,-0.322550,0.660732,0.928335,0.259853,0.026575,-0.450223
2,-0.880518,0.653001,-0.957072,-0.967709,-0.744758,-0.18532,-0.086472,-0.671749,-0.17253,0.315448,-0.018675,0.424262,-0.762185,0.190906,-0.491139
3,-0.880518,1.240795,0.021795,1.033369,0.942033,-0.18532,-0.086472,-0.671749,-0.17253,0.543304,0.298382,-0.163824,-0.257293,-0.137755,-0.491139
4,1.135695,-1.580613,1.979529,-0.967709,-0.744758,-0.18532,-0.086472,-0.671749,-0.17253,-0.595978,-0.924551,0.214231,-0.931299,-0.384252,-0.204725


In [27]:
pca = PCA(0.9)  # we want 90% of the information is attributed by principal components
pca.fit(x_train_sd)

PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [28]:
pca.n_components_ # number of principal components

11

Then we take a look at the ratios of explained variance, which state that how much information (variance) can be accounted by each of the principal components.

In [29]:
np.round(pca.explained_variance_ratio_,3)

array([0.217, 0.128, 0.102, 0.075, 0.07 , 0.069, 0.067, 0.058, 0.052,
       0.047, 0.038])

In [30]:
round(sum(pca.explained_variance_ratio_),3) 
# 92.3% of the information is attributed by principal components

0.923

In [31]:
pc = pca.fit_transform(x_train_sd) # Principal components
df_pc = pd.DataFrame(data = pc, columns = ['pc1','pc2','pc3','pc4','pc5','pc6',
                                           'pc7','pc8','pc9','pc10','pc11'])
df_pc.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,pc11
0,-0.938280,-0.019774,-0.016178,-1.547015,0.320923,-0.122549,0.651331,0.238001,0.332116,-0.134682,-0.135853
1,0.760479,0.829974,-0.825928,-0.914624,-0.217050,-0.752348,1.173505,-0.570348,-0.056062,-1.416116,0.940763
2,0.202428,-1.428618,-0.628125,-0.646743,-0.383793,0.902968,0.186972,-0.307763,-0.360890,0.209880,0.405470
3,-0.267417,0.668644,-0.262321,-0.454233,0.151922,1.486881,-0.561270,-0.170430,-0.441784,-0.619568,-1.010428
4,-1.646330,-0.700462,-0.152781,0.696984,1.038738,-2.052524,-0.711680,0.680242,-0.076988,0.776347,1.115150


Perform dimension reduction (feature selection) by the principal components we've found.

In [32]:
x_train_pc = pca.transform(x_train_sd)
x_test_pc = pca.transform(x_test_sd) # The features we're going to put in the logistic regression model

### Apply logistic regression to `x_test_pc`

In [33]:
lr_pca = LogisticRegression(solver = 'lbfgs')
lr_pca.fit(x_train_pc, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)

In [34]:
y_pred_pca = lr_pca.predict(x_test_pc)

To compare the predicted results with the original test data, we need a confusion matrix, which is defined as:

###   <center> Confusion Matrix </center>
|           |Observe YES      |Observe No        |
|-----------|-----------------|------------------|
|Predict YES|True Positive  (TP)|False Positive  (FP)|
|Predict  NO|False Negative (FN)|True Negative   (TN)|

In [183]:
def pred_result(y_test, y_pred):
    y_test = np.array(y_test)
    y_pred = np.array(y_pred)
    cnt_tp = 0; cnt_fp = 0; cnt_fn = 0; cnt_tn = 0
    
    for i in range(len(y_test)):
        if (y_test[i] == 1 and y_pred[i] == 1):
            cnt_tp += 1 
        elif (y_test[i] == 0 and y_pred[i] == 1):
            cnt_fp += 1 
        elif (y_test[i] == 1 and y_pred[i] == 0):
            cnt_fn += 1
        elif (y_test[i] == 0 and y_pred[i] == 0):
            cnt_tn += 1

    matrix = np.array([[cnt_tp,cnt_fp],[cnt_fn,cnt_tn]])
    accuracy = (cnt_tp+cnt_tn)/(cnt_fp+cnt_fn+cnt_tp+cnt_tn)
    print("The confusion matrix is\n")
    print(matrix)
    print("\n")
    print("The accuracy is")
    return round(accuracy, 4)

The prediction result of PCA model.

In [36]:
pred_result(y_test, y_pred_pca)

The confusion matrix is

[[ 11   2]
 [162 923]]


The acuracy is


0.8506

### LASSO

The cost function of LASSO:
$$\sum_{i=1}^{n}(Y_{i}\sum_{j=1}^{p} X_{i,j}\beta_{j})^2 + \lambda\sum_{j=1}^{p}\mid\beta_{j}\mid$$

First of all, we need to determine the regularization parameter $\alpha$ for our lasso model. When $\alpha=0$, the lasso model gives the same coefficients as a linear regression. The higher the $\alpha$, most of the feature coefficients shrink to 0. 

In [179]:
c = np.linspace(0.01,1,100)
A = []
for k in range(len(c)):
    clf = LogisticRegression(penalty="l1", solver='saga', C=c[k])
    clf.fit(x_train_sd, y_train)
    y_pred_lasso = clf.predict(x_test_sd)
    A.append(pred_result(y_test, y_pred_lasso))
A.index(max(A))

15

As a result, we choose $C=0.16$ for the lasso regression.

In [195]:
clf = LogisticRegression(penalty="l1", solver='saga', C=0.16)
clf.fit(x_train_sd, y_train)
y_pred_lasso = clf.predict(x_test_sd)

We take a look at the coefficients of our shrinkage model. 

In [186]:
np.round(clf.coef_,4)

array([[ 2.757e-01,  5.555e-01, -3.040e-02,  0.000e+00,  1.981e-01,
         0.000e+00,  6.340e-02,  1.172e-01,  0.000e+00,  1.085e-01,
         2.875e-01,  0.000e+00, -1.000e-04,  0.000e+00,  1.384e-01]])

Variable `currentSmoker`, `BPMeds`, `diabetes`, `diaBP`, `heartRate` are shrunk to 0.

The prediction result of the lasso model.

In [200]:
pred_result(y_test, y_pred_lasso)

The confusion matrix is

[[ 12   1]
 [161 924]]


The accuracy is


0.8525

### Elastic Net

In [196]:
clf_elastic = LogisticRegression(penalty="elasticnet", solver='saga', C=0.16)
clf_elastic.fit(x_train_sd, y_train)
y_pred_elastic = clf_elastic.predict(x_test_sd)

We take a look at the coefficients of the elastic net model, which do not shrink to 0.

In [198]:
np.round(clf_elastic.coef_,4)

array([[ 0.297 ,  0.5587, -0.0532, -0.0175,  0.222 ,  0.0093,  0.0728,
         0.1338, -0.0225,  0.1278,  0.2944, -0.0072, -0.0297,  0.0032,
         0.1653]])

The prediction result of the elastic model.

In [199]:
pred_result(y_test, y_pred_elastic)

The confusion matrix is

[[ 11   3]
 [162 922]]


The accuracy is


0.8497

The elastic net is not a good model for prediction, since its accuracy score is equal to the original logistic regression that does not perform feature selection.

In [201]:
lr = LogisticRegression()
lr.fit(x_train_sd, y_train)
y_pred_lr = clf_elastic.predict(x_test_sd)

In [202]:
pred_result(y_test, y_pred_lr)

The confusion matrix is

[[ 11   3]
 [162 922]]


The accuracy is


0.8497

### K-Means

In [210]:
from sklearn import cluster
clf_kmeans = cluster.KMeans(init='k-means++', n_clusters=2, random_state=100)
clf_kmeans.fit(x_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=100, tol=0.0001, verbose=0)

In [211]:
y_pred_kmeans = clf_kmeans.predict(x_test)

The prediction result of Kmeans.

In [213]:
pred_result(y_test, y_pred_kmeans)

The confusion matrix is

[[ 95 395]
 [ 78 530]]


The accuracy is


0.5692

### Support vector machine (SVM) 

In [239]:
svc = svm.SVC(gamma = 0.001, C = 10, kernel='linear')
svc.fit(x_train, y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [240]:
y_pred_svc = svc.predict(x_test)

In [241]:
pred_result(y_test, y_pred_svc)

The confusion matrix is

[[  9   2]
 [164 923]]


The accuracy is


0.8488

### Discussion

 1. Since we aim to improve the prediction on `TenYearCHD`, we did not perform cross-validaton for searching the $\alpha$ that resulted in the smallest MSE. 
 2. Compared to the original logistic regression, PCA and LASSO successfully performs dimension reduction (from 15 to 11) and improves the accuracy of prediction.
 3. LASSO performs the best in predicting `TenYearCHD` for our test data. 
 4. Elastic net model does not perform feature selection, therefore the it results in the same prediction result as the original logistic regression `lr`.
 5. It is reasonable that KMeans performs the worst since it does not need target value(`TenYearCHD`) when classifying.
 6. SVM is slightly worse than the original logistic regression.